In [1]:
import os
import time
import random
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
LAST_PROCESSED_FILE = "last_processed.txt"
CSV_FILE = "member_usernames.csv"

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 20)
    return driver, wait

def get_last_processed():
    if os.path.exists(LAST_PROCESSED_FILE):
        with open(LAST_PROCESSED_FILE, "r", encoding="utf-8") as file:
            return file.read().strip() or None
    return None

def save_last_processed(username):
    with open(LAST_PROCESSED_FILE, "w", encoding="utf-8") as file:
        file.write(username)

def save_usernames_to_csv(usernames):
    df = pd.DataFrame({'Username': list(usernames)})
    df.to_csv(CSV_FILE, mode='a', header=not os.path.exists(CSV_FILE), index=False, encoding='utf-8')
    print(f"✅ {len(usernames)} نام کاربری جدید ذخیره شد!")

def open_group(driver, wait, index):
    try:
        chat_list = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.chatlist-chat')))
        chat_list[index].click()
        print(f"✅ گروه {index + 1} باز شد!")
        time.sleep(random.uniform(2, 5))
        return True
    except Exception as e:
        print(f"❌ خطا در باز کردن گروه {index + 1}: {e}")
        return False

def open_group_info(driver, wait):
    try:
        chat_info_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.chat-info .person')))
        chat_info_element.click()
        print("✅ اطلاعات گروه باز شد!")
        time.sleep(random.uniform(2, 5))
        return True
    except Exception as e:
        print(f"❌ خطا در باز کردن اطلاعات گروه: {e}")
        return False

def is_in_group_info_page(driver, wait):
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.search-super-content-container')))
        return True
    except:
        return False

def ensure_in_group_info(driver, wait):
    if not is_in_group_info_page(driver, wait):
        print("⚠️ از صفحه اطلاعات گروه خارج شدیم! بازگشت...")
        driver.back()
        time.sleep(random.uniform(3, 7))
        return open_group_info(driver, wait)
    return True

def extract_members(driver, wait):
    if not open_group(driver, wait, 0) or not open_group_info(driver, wait):
        return False

    usernames = set()
    last_processed = get_last_processed()
    start_processing = last_processed is None  
    actions = ActionChains(driver)
    last_count, scroll_attempts = -1, 0
    max_scroll_attempts = 1000

    while True:
        if not ensure_in_group_info(driver, wait):
            return False

        try:
            members = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.search-super-content-container .row-clickable')))
        except Exception as e:
            print(f"❌ خطا در دریافت لیست اعضا: {e}")
            break

        if len(members) == last_count:
            scroll_attempts += 1
            if scroll_attempts >= max_scroll_attempts:
                print("✅ تمام اعضا بررسی شدند!")
                break
        else:
            scroll_attempts = 0  
        last_count = len(members)

        for i, member in enumerate(members):
            if i < 2:
                continue  

            try:
                actions.move_to_element(member).perform()
                time.sleep(random.uniform(1, 5))
                member.click()
                print(f"🔎 در حال بررسی عضو {i + 1}")
                time.sleep(random.uniform(3, 7))

                if not ensure_in_group_info(driver, wait):
                    return False

                try:
                    info_sections = wait.until(EC.presence_of_all_elements_located(
                        (By.XPATH, '//div[contains(@class, "row-clickable") and contains(@class, "row-with-padding")]')))
                    time.sleep(random.uniform(1, 5))
                    found_username = False
                    for section in info_sections:
                        try:
                            label = section.find_element(By.CLASS_NAME, "i18n").text.strip()
                            if label == "Username":
                                username = section.find_element(By.CLASS_NAME, "row-title").text.strip()
                                username = re.sub(r"[^\w@_.]", "", username)
                                if username:
                                    if last_processed and username == last_processed:
                                        start_processing = True  
                                    if start_processing and username not in usernames:
                                        usernames.add(f"@{username}")
                                        save_last_processed(username)
                                        print(f"✅ نام کاربری پیدا شد: @{username}")
                                found_username = True
                                break
                        except:
                            continue
                    if not found_username:
                        print("⚠️ نام کاربری یافت نشد.")
                except Exception as e:
                    print(f"❌ خطا در یافتن نام کاربری برای عضو {i + 1}: {e}")

                driver.back()
                time.sleep(random.uniform(2, 5))

                if not ensure_in_group_info(driver, wait):
                    return False

                if len(usernames) % 10 == 0:  
                    save_usernames_to_csv(usernames)
                    usernames.clear()

            except Exception as e:
                print(f"❌ خطا در پردازش عضو {i + 1}: {e}")
                continue

        driver.execute_script("arguments[0].scrollIntoView(true);", members[-1])
        time.sleep(random.uniform(5, 10))
        if scroll_attempts >= 5:
            print("🔄 رفرش صفحه برای بارگذاری اعضای جدید...")
            driver.refresh()
            time.sleep(random.uniform(5, 10))
            if not open_group(driver, wait, 0) or not open_group_info(driver, wait):
                return False
            scroll_attempts = 0

    if usernames:
        save_usernames_to_csv(usernames)

    return bool(usernames)

if __name__ == "__main__":
    driver, wait = setup_driver()
    driver.get("https://web.telegram.org/")
    time.sleep(random.uniform(5, 10))
    input('برای ادامه دکمه Enter را بزنید')

    if extract_members(driver, wait):
        print("✅ عملیات با موفقیت انجام شد!")
    else:
        print("❌ عملیات ناموفق بود.")

    driver.quit()


برای ادامه دکمه Enter را بزنید 


✅ گروه 1 باز شد!
✅ اطلاعات گروه باز شد!
🔎 در حال بررسی عضو 3
⚠️ نام کاربری یافت نشد.
✅ 0 نام کاربری جدید ذخیره شد!
🔎 در حال بررسی عضو 4
✅ نام کاربری پیدا شد: @ashyane1990
🔎 در حال بررسی عضو 5
✅ نام کاربری پیدا شد: @Himyladys
🔎 در حال بررسی عضو 6
✅ نام کاربری پیدا شد: @Gggjhggio
🔎 در حال بررسی عضو 7
✅ نام کاربری پیدا شد: @Dspbad
🔎 در حال بررسی عضو 8
✅ نام کاربری پیدا شد: @mmomdh
🔎 در حال بررسی عضو 9
⚠️ نام کاربری یافت نشد.
🔎 در حال بررسی عضو 10
⚠️ نام کاربری یافت نشد.
🔎 در حال بررسی عضو 11
⚠️ نام کاربری یافت نشد.
🔎 در حال بررسی عضو 12
✅ نام کاربری پیدا شد: @Aligh7980
🔎 در حال بررسی عضو 13
⚠️ نام کاربری یافت نشد.
🔎 در حال بررسی عضو 14
⚠️ نام کاربری یافت نشد.
🔎 در حال بررسی عضو 15
⚠️ نام کاربری یافت نشد.
🔎 در حال بررسی عضو 16
✅ نام کاربری پیدا شد: @Naserazzar
🔎 در حال بررسی عضو 17
⚠️ نام کاربری یافت نشد.
🔎 در حال بررسی عضو 18
⚠️ نام کاربری یافت نشد.
🔎 در حال بررسی عضو 19
⚠️ نام کاربری یافت نشد.
🔎 در حال بررسی عضو 20
✅ نام کاربری پیدا شد: @Jamshid0897sinapor
🔎 در حال بررسی عضو 21
⚠️ نام کا

In [ ]:
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 20)
    return driver, wait
    
driver, wait = setup_driver()
driver.get("https://web.telegram.org/")
time.sleep(random.uniform(2, 3))
input('برای ادامه دکمه Enter را بزنید')

def open_group(index):
    try:
        chat_list = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.chatlist-chat')))
        chat_list[index].click()
        print(f"✅ گروه {index + 1} باز شد!")
        time.sleep(random.uniform(2, 3))
        return True
    except Exception as e:
        print(f"❌ خطا در باز کردن گروه {index + 1}: {e}")
        return False

def open_group_info():
    try:
        chat_info_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.chat-info .person')))
        chat_info_element.click()
        print("✅ اطلاعات گروه باز شد!")
        time.sleep(random.uniform(2, 3))
        return True
    except Exception as e:
        print(f"❌ خطا در باز کردن اطلاعات گروه: {e}")
        return False

def click_add_button():
    try:
        add_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'btn-circle') and contains(@class, 'btn-corner')]//span[contains(@class, 'tgico') and text()='']")))
        driver.execute_script("arguments[0].click();", add_button)
        print("✅ دکمه افزودن کلیک شد!")
        time.sleep(random.uniform(2, 5))
    except Exception as e:
        print(f"❌ خطا در کلیک روی دکمه افزودن: {e}")

def confirm_addition():
    try:
        add_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'add-members-container')]//button[contains(@class, 'btn-circle') and contains(@class, 'btn-corner')]")))
        driver.execute_script("arguments[0].click();", add_button)
        
        add_confirm_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'popup-buttons')]/button[1]")))
        driver.execute_script("arguments[0].click();", add_confirm_button)
        
        print("✅ دکمه تایید کلیک شد!")
        time.sleep(random.uniform(1, 7))
    except Exception as e:
        print(f"❌ خطا در کلیک روی دکمه تایید: {e}")
        driver.refresh()
        time.sleep(random.uniform(2, 4))
        confirm_addition()

def refresh_page():
    sleep_time = random.uniform(20, 100)  # مقداردهی متغیر
    print("🔄 در حال رفرش صفحه برای ادامه فرآیند...")
    print(f"⏳ مکث قبل از رفرش به مدت {sleep_time:.2f} ثانیه")
    time.sleep(sleep_time)  # صبر قبل از رفرش
    driver.refresh()
    time.sleep(random.uniform(5, 10))  # صبر بعد از رفرش


def add_members_to_group():
    while True:
        try:
            df = pd.read_csv("member_usernames.csv")
            usernames = df["Username"].tolist()
        except Exception as e:
            print(f"❌ خطا در خواندن نام‌های کاربری از CSV: {e}")
            return
        
        if not usernames:
            print("🎉 تمامی اعضا اضافه شدند!")
            break
        
        if not open_group(1):
            continue
        if not open_group_info():
            continue
        click_add_button()
        
        for username in usernames:
            try:
                search_box = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.selector-search-input')))
                search_box.click()
                time.sleep(1)
                search_box.send_keys(Keys.CONTROL + "a")
                search_box.send_keys(Keys.BACKSPACE)
                time.sleep(1)
                search_box.send_keys(username)
                time.sleep(1)
                search_box.send_keys(Keys.ENTER)
                time.sleep(random.uniform(2, 4))
                
                results = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.selector-list-section-content .chatlist')))
                
                if results:
                    results[0].click()
                    time.sleep(random.uniform(2, 6))
                    print(f"✅ {username} انتخاب شد!")
                else:
                    print(f"⚠️ یوزرنیم {username} یافت نشد!")
                
                confirm_addition()
                print(f"✅ {username} با موفقیت اضافه شد!")
                df = df.iloc[1:]
                df.to_csv("member_usernames.csv", index=False)
                print(f"🗑️ {username} از فایل CSV حذف شد!")
                refresh_page()
                
                break  # بعد از رفرش، از ابتدا شروع کند
            except Exception as e:
                print(f"❌ خطا در انتخاب {username}: {e}")

add_members_to_group()
